In [1]:
import pandas as pd
import re
from unidecode import unidecode
import json
from typing import Dict,List
import os 

In [2]:
data_raw=[]
test_data_raw=[]
pattern=re.compile(r"https://www\.rottentomatoes\.com/m/([^/]+)")
grade_exception=set()
grade_map = {
    "F-": 0,
    "F": 1,
    "F+": 2,
    "D-": 3,
    "D": 4,
    "D+": 5,
    "C-": 6,
    "C": 7,
    "C+": 8,
    "B-": 9,
    "B": 10,
    "B+": 11,
    "A-": 12,
    "A": 13,
    "A+": 14
}
def parseGrade(grade:str):
    if grade is None:
        return None
    grade=grade.replace(" ","")
    try:
        grade=float(grade)
        max_grade=1
        while(max_grade<grade):
            max_grade*=10
        return grade/max_grade
    except:
        pass
    num_score=re.search(r"(?P<first>(\d+\.*\d*)|(\.\d+))/(?P<second>(\d+\.*\d*)|(\.\d+))",grade)
    if num_score and num_score.group("first") and num_score.group("second"):
        try:
            return float(num_score.group("first"))/float(num_score.group("second"))
        except:
            return grade
    alphabet_score=re.search(r"(\w) *(\+|-)*",grade)
    if alphabet_score and alphabet_score.group(1):
        try:
            submark=alphabet_score.group(2)
        except:
            pass
        if not submark:
            submark=""
        _abcgrade=alphabet_score.group(1).upper()+submark
        if _abcgrade in grade_map:
            return grade_map[_abcgrade]/grade_map["A+"]
    grade_exception.add(grade)
    # print(f"\{grade}\\")
    return grade
with open("review_list","r") as f:
    for line in f:
        movie:Dict=json.loads(line)
        for k,v in movie.items():
            movie=pattern.search(k)
            if movie is not None:
                movie=movie[1]
            _raw=[]
            for review in v:
                additional:List[str]=[r.strip() for r in review["additional"].split("|")]
                grade,date=None,None
                if len(additional)==3:
                    try:
                        _grade=re.search(r"Original Score: +(.+)",additional[1]).group(1)
                        grade=parseGrade(_grade)
                        if(grade and type(grade)!= str and grade>1 ):
                            # print("Discard parsed",_grade)
                            grade=_grade
                        date=additional[2]
                    except:
                        date=additional[2]
                        print(date)
                elif len(additional)==2:
                    date=additional[1]
                else:
                    raise NotImplemented
                review.update({"date":date,"grade":grade})
                _raw.append(dict(review,**{"movie":movie}))
            if len(_raw)>1:
                test_data_raw.append(_raw[-1])
                _raw=_raw[:-1]
            data_raw.extend(_raw)

In [3]:
df=pd.DataFrame(data_raw)
df

,critic,publication,state,review,additional,date,grade,movie
0,Manori Ravindran,National Post,fresh,There's an honesty and whip smart humour to th...,"Full Review | Original Score: 3/4 | Aug 16, 2013","Aug 16, 2013",0.75,everyday_is_like_sunday
1,Ruth Maramis,FlixChatter Film Blog,rotten,The reunion story set in the ancient Egyptian ...,"Full Review | Original Score: 2.5/5 | Mar 3, 2023","Mar 3, 2023",0.5,luxor
2,Victoria Luxford,City AM,fresh,Luxors thoughtful tone and leisurely pace wont...,"Full Review | Original Score: 4/5 | Feb 26, 2022","Feb 26, 2022",0.8,luxor
3,Sandie Angulo Chen,AWFJ.org,fresh,"Andrea Riseborough is a brilliant actress, and...","Full Review | Jan 10, 2022","Jan 10, 2022",None,luxor
4,Cate Marquis,AWFJ.org,fresh,"...an intriguing reflection amid the ruins, on...","Full Review | Jan 10, 2022","Jan 10, 2022",None,luxor
...,...,...,...,...,...,...,...,...
356242,J. Hoberman,Village Voice,fresh,It's a measure of the movie's success that one...,"Full Review | Jan 4, 2011","Jan 4, 2011",None,if_i_want_to_whistle_i_whistle
356243,Louis Proyect,rec.arts.movies.reviews,fresh,"Despite its dire economic circumstances, Roman...","Full Review | Jan 3, 2011","Jan 3, 2011",None,if_i_want_to_whistle_i_whistle
356244,Tim Brayton,Antagony & Ecstasy,rotten,A stilted and frustrating grab-bag of stylisti...,"Full Review | Original Score: 5/10 | Oct 13, 2010","Oct 13, 2010",0.5,if_i_want_to_whistle_i_whistle
356245,Leslie Felperin,Hollywood Reporter,fresh,The two lead actors... bring nuance and commit...,"Full Review | Sep 18, 2021","Sep 18, 2021",None,what_happened_to_the_wolf


In [4]:
test_df=pd.DataFrame(test_data_raw)
test_df

,critic,publication,state,review,additional,date,grade,movie
0,Matt Cipolla,The Spool,rotten,[Riseborough gives] a deeply empathic performa...,"Full Review | Jan 24, 2020","Jan 24, 2020",None,luxor
1,Oz,eFilmCritic.com,fresh,"In Junebug, everything is great - no question ...","Full Review | Original Score: 5/5 | Feb 14, 2005","Feb 14, 2005",1.0,junebug
2,Ken Eisner,Variety,rotten,Artist-turned-helmer Su Rynard gets the tough ...,"Full Review | Oct 13, 2006","Oct 13, 2006",None,kardia
3,Ian Mantgani,UK Critic,fresh,,"Full Review | Original Score: 8/10 | Jan 1, 2000","Jan 1, 2000",0.8,life
4,Jim Vejvoda,IGN Movies,fresh,Thor may not be a game-changer for comic book ...,"Full Review | Original Score: 3.5/5 | Apr 17, ...","Apr 17, 2011",0.7,thor
...,...,...,...,...,...,...,...,...
4549,Andrew Barker,Variety,fresh,[The film] gives the seminal early '90s hip-ho...,"Full Review | Jan 25, 2011","Jan 25, 2011",None,beats_rhymes_and_life_the_travels_of_a_tribe_c...
4550,Roger Moore,Movie Nation,rotten,"Like ""Tucker and Dale vs. Evil,"" without the e...","Full Review | Original Score: 1/4 | Apr 13, 2018","Apr 13, 2018",0.25,the_escape_of_prisoner_614
4551,Wendy Ide,Screen International,fresh,A film which celebrates empowerment and the ex...,"Full Review | Jul 17, 2021","Jul 17, 2021",None,casablanca_beats
4552,Guy Lodge,Variety,fresh,Gemma Arterton's strong but shattered performa...,"Full Review | Sep 11, 2017","Sep 11, 2017",None,the_escape_2018


In [5]:
pattern=re.compile(r"https://www\.rottentomatoes\.com/m/([^/]+)")
metadatas=[]
with open("movie_list_enriched","r") as f:
    for line in f:
        meta:Dict=json.loads(line)
        for k,v in meta.items():
            title=pattern.findall(k)
            if not title:
                continue
            title=title[0]
            submeta=v.pop("meta",None)
            temp=dict(v,**{"movie":title})
            if submeta:
                for item in submeta:
                    descriptions=re.findall(r"[^:]+",item)
                    if len(descriptions)>=2:
                        temp.update({descriptions[0]:' '.join(descriptions[1:])})
                    else:
                        print(item)
            metadatas.append(temp)


In [6]:
metadf=pd.DataFrame(metadatas)
metadf=metadf[["movie","description","Genre","Original Language","Director","Runtime"]]
metadf=metadf.drop_duplicates(["movie"])
metadf=metadf[metadf["Original Language"].str.contains("English",na=False)]
metadf

,movie,description,Genre,Original Language,Director,Runtime
0,everyday_is_like_sunday,"Three Toronto roommates (David Dineen-Porter, ...","Comedy, Drama",English,Pavan Moondi,1h 30m
1,luxor,When British aid worker Hana returns to the an...,"Drama, Romance",English (United Kingdom),Zeina Durra,1h 25m
2,junebug,On the way to meet with an independent artist ...,"Comedy, Drama",English,Phil Morrison,1h 46m
3,kardia,Hope (Mimi Kuzyk) must unlock memories from he...,Drama,English,Su Rynard,1h 25m
4,life,"During Prohibition, loudmouth Harlem grifter R...","Comedy, Drama",English,Ted Demme,1h 48m
...,...,...,...,...,...,...
4996,evil_alien_conquerors,"Two extraterrestrials (Diedrich Bader, Chris P...","Comedy, Sci-fi",English,Chris Matheson,1h 29m
4997,beats_rhymes_and_life_the_travels_of_a_tribe_c...,Actor Michael Rapaport examines the music of t...,"Documentary, Music",English,Michael Rapaport,1h 37m
4998,the_escape_of_prisoner_614,After two deputy sheriffs lose their jobs beca...,"Western, Comedy",English,Zach Golden,1h 35m
5000,the_escape_2018,Tara is a housewife in suburban London who's r...,Drama,English,Dominic Savage,1h 41m


In [7]:
metadfview=metadf.__deepcopy__()
dfview=df.merge(metadfview[["Original Language","movie"]],how="inner",on="movie").drop(["Original Language"],axis=1)
os.makedirs("filtered",exist_ok=True)
dfview.to_csv("./filtered/review_list")
metadfview.to_csv("./filtered/metadata")

In [8]:
metadfview=metadf.__deepcopy__()
dfview=test_df.merge(metadfview[["Original Language","movie"]],how="inner",on="movie").drop(["Original Language"],axis=1)
os.makedirs("filtered_test",exist_ok=True)
dfview.to_csv("./filtered_test/review_list")
metadfview.to_csv("./filtered_test/metadata")